In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install dspy-ai
!pip install datasets
!pip install python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.7/261.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 31.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 999.9/999.9 kB 20.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
  Attempting uninstall: openai
    Found existing installation: openai 1.97.1
    Uninstalling openai-1.97.1:
      Successfully uninstalled openai-1.9

In [2]:
API_KEY = "sk-proj------"


In [4]:
import os
import multiprocessing
from datasets import load_dataset
import dspy

#  Initialize LMs

In [5]:

lm = dspy.LM(
    model="gpt-4o-mini",
    provider="openai",
    api_key=API_KEY
)
dspy.settings.configure(lm=lm)


# Reflection LM

In [6]:
reflection_lm = dspy.LM(
    model="gpt-5",
    provider="openai",
    temperature=1.0,
    max_tokens=32000,
    api_key=API_KEY
)


# Load Dataset

In [7]:
dataset = load_dataset("openai/openai_humaneval")
problems = dataset["test"]

print("Number of problems:", len(problems))
print("Example problem:\n", problems[0]["prompt"])

# Use only first 20 tasks as training data
subset = problems.select(range(20))


README.md: 0.00B [00:00, ?B/s]

openai_humaneval/test-00000-of-00001.par(…):   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

Number of problems: 164
Example problem:
 from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """



# DSPy module: Generate a useful programming prompt

In [8]:

class GeneratePrompt(dspy.Module):
    def __init__(self):
        super().__init__()
        # we want the LM to predict an optimized instruction-style prompt
        self.generate = dspy.Predict("task_description -> optimized_prompt")

    def forward(self, task_description):
        return self.generate(task_description=task_description)


#  Prepare training data

In [9]:

trainset = []
for item in subset:
    task_text = item["prompt"]
    # we’ll define the “optimized prompt” target as instruction-style text (for training)
    gold_prompt = (
        "Write a Python function for the following problem.\n"
        f"{task_text.strip()}\n"
        "Requirements:\n"
        "1. Include type hints.\n"
        "2. Handle edge cases.\n"
        "3. Return the correct output."
    )
    ex = dspy.Example(
        task_description=task_text,
        optimized_prompt=gold_prompt
    ).with_inputs("task_description")
    trainset.append(ex)

print(f"Training examples prepared: {len(trainset)}")


Training examples prepared: 20


In [12]:
def prompt_similarity_metric(gold, pred, trace=None, pred_name=None, pred_trace=None):
    gold_text = getattr(gold, "optimized_prompt", gold)
    pred_text = getattr(pred, "optimized_prompt", pred)

    gold_text = str(gold_text).lower()
    pred_text = str(pred_text).lower()

    # Simple lexical overlap metric
    overlap = len(set(gold_text.split()) & set(pred_text.split()))
    total = max(len(set(gold_text.split())), 1)
    return overlap / total

# Initialize GEPA optimizer

In [13]:
# 7. Initialize GEPA optimizer
gepa = dspy.GEPA(
    metric=prompt_similarity_metric,
    max_full_evals=3,
    reflection_lm=reflection_lm
)

In [14]:



# 8. Compile model and optimize
generator = GeneratePrompt()
gepa.compile(generator, trainset=trainset)


2025/10/20 03:51:58 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 60 metric calls of the program. This amounts to 3.00 full evals on the train set.
2025/10/20 03:51:58 INFO dspy.teleprompt.gepa.gepa: Using 20 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
GEPA Optimization:   0%|          | 0/60 [00:00<?, ?rollouts/s]2025/10/20 03:52:05 INFO dspy.evaluate.evaluate: Average Metric: 6.667611573150414 / 20 (33.3%)
2025/10/20 03:52:05 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.3333805786575207
GEPA Optimization:  33%|███▎      | 20/60 [00:06<00:13,  2.96rollouts/s]2025/10/20 03:52:05 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.3333805786575207


Average Metric: 1.01 / 3 (33.8%): 100%|██████████| 3/3 [00:00<00:00, 41.09it/s]

2025/10/20 03:52:05 INFO dspy.evaluate.evaluate: Average Metric: 1.012800686096856 / 3 (33.8%)


2025/10/20 03:52:35 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for generate: Instruction: Generate an optimized, self-contained implementation prompt from the given task_description.

Goal
- Transform the provided Python task_description (typically a function stub with a docstring and doctest examples) into a clear, precise prompt that a coding model can follow to implement the required function(s) correctly.

General Requirements
- Target language: Python 3.
- Use only the standard library unless the task explicitly says otherwise.
- Preserve all function names, signatures, and type hints exactly as given.
- If multiple functions are defined or referenced, instruct to implement all functions that are:
  - Declared in the snippet and have non-trivial behavior implied by the docstring(s), and/or
  - Required by the examples or by other functions in the snippet.
- Restate all behavioral requirements explicitly. Do not rely on the original docstring being available lat

Average Metric: 1.38 / 3 (45.9%): 100%|██████████| 3/3 [00:00<00:00, 45.08it/s]

...

2025/10/20 03:53:31 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Proposed new text for generate: Instruction: Generate an optimized, self-contained implementation prompt from the given task_description.

Goal
- Transform the provided Python task_description (typically a function stub with a docstring and doctest examples) into a clear, precise prompt that a coding model can follow to implement the required function(s) correctly.

General Requirements
- Target language: Python 3.
- Use only the standard library unless the task explicitly says otherwise.
- Preserve all function names, signatures, and type hints exactly as given (no renaming, no signature changes).
- If multiple functions are defined or referenced, instruct to implement all functions that are:
  - Declared in the snippet and have non-trivial behavior implied by the docstring(s), and/or
  - Required by the examples, or called by other functions in the snippet.
- Restate all behavioral requirements explicitly. Do not rely o

Average Metric: 1.23 / 3 (41.0%): 100%|██████████| 3/3 [00:00<00:00, 44.79it/s]

2025/10/20 03:53:33 INFO dspy.evaluate.evaluate: Average Metric: 1.2286079182630907 / 3 (41.0%)


2025/10/20 03:54:12 INFO dspy.teleprompt.gepa.gepa: Iteration 3: Proposed new text for generate: Instruction: Generate an optimized, self-contained implementation prompt from the given task_description.

Goal
- Transform the provided Python task_description (usually a function stub with a docstring and doctest examples) into a clear, precise prompt that a coding model can follow to implement the required function(s) correctly.

General Requirements
- Target language: Python 3.
- Use only the standard library unless the task explicitly says otherwise.
- Preserve all function names, signatures, and type hints exactly as given (copy the full signature(s) verbatim).
- If multiple functions are declared or implied by examples, instruct to implement all functions that:
  - Are declared in the snippet and have non-trivial behavior implied by the docstring(s), and/or
  - Are required by examples or by other functions in the snippet.
- Restate all behavioral requirements explicitly so the solut

generate = Predict(StringSignature(task_description -> optimized_prompt
    instructions="Instruction: Generate an optimized, self-contained implementation prompt from the given task_description.\n\nGoal\n- Transform the provided Python task_description (usually a function stub with a docstring and doctest examples) into a clear, precise prompt that a coding model can follow to implement the required function(s) correctly.\n\nGeneral Requirements\n- Target language: Python 3.\n- Use only the standard library unless the task explicitly says otherwise.\n- Preserve all function names, signatures, and type hints exactly as given (copy the full signature(s) verbatim).\n- If multiple functions are declared or implied by examples, instruct to implement all functions that:\n  - Are declared in the snippet and have non-trivial behavior implied by the docstring(s), and/or\n  - Are required by examples or by other functions in the snippet.\n- Restate all behavioral requirements explicitly so the 

# Test

In [15]:
# 9. Test with a few new HumanEval problems
print("\n--- Testing optimized prompt generator ---")
test_examples = problems.select(range(20, 23))
for ex in test_examples:
    result = generator(task_description=ex["prompt"])
    print(f"\n[Task Description]\n{ex['prompt']}")
    print(f"[Generated Optimized Prompt]\n{result.optimized_prompt}")



--- Testing optimized prompt generator ---

[Task Description]
from typing import List, Tuple


def find_closest_elements(numbers: List[float]) -> Tuple[float, float]:
    """ From a supplied list of numbers (of length at least two) select and return two that are the closest to each
    other and return them in order (smaller number, larger number).
    >>> find_closest_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.2])
    (2.0, 2.2)
    >>> find_closest_elements([1.0, 2.0, 3.0, 4.0, 5.0, 2.0])
    (2.0, 2.0)
    """

[Generated Optimized Prompt]
Implement the function `find_closest_elements` that takes a list of float numbers and returns a tuple containing the two closest numbers in ascending order. Ensure the function handles cases where the closest numbers are the same. Include examples in the docstring to demonstrate the expected behavior.

[Task Description]
from typing import List


def rescale_to_unit(numbers: List[float]) -> List[float]:
    """ Given list of numbers (of at least two e

# optimized system prompt template

In [16]:


# 10. Extract optimized system prompt template
optimized_prompt = generator.generate.prompt_template
print("\n--- Final Optimized DSPy Prompt Template ---")
print(optimized_prompt)

AttributeError: 'Predict' object has no attribute 'prompt_template'